# Historical Conversions Analysis

Import Libraries & Data

In [1]:
import numpy as np
import pandas
import datetime

In [2]:
# df = pandas.read_csv('data/Summer_Peak.csv')
df_1516 = pandas.read_csv('data/Summer_Peak_15.16.csv')
df_1617 = pandas.read_csv('data/Summer_Peak_16.17.csv')
df_1718 = pandas.read_csv('data/Summer_Peak_17.18.csv')
df_1819 = pandas.read_csv('data/Summer_Peak_18.19.csv')

In [3]:
def conversion_rates(data, year=None, month=None, day = None):
    df_adjusted_date = data.copy()
    df_adjusted_date['Date realized'] = pandas.to_datetime(df_adjusted_date['Date realized'])
    if year != None and month != None and day != None:
        df_realized = df_adjusted_date.loc[(df_adjusted_date['Date realized'] <= pandas.Timestamp(year=year,month=month,day=day))]
    else:
        df_realized = df_adjusted_date.loc[(df_adjusted_date['Date realized'] != '')]
    df_realized_grouped = df_realized.groupby(by=['Home MC','Host MC']).count()[['EP ID']]

    df_approved_grouped = df_adjusted_date.groupby(by=['Home MC','Host MC']).count()[['EP ID']]
    df_conversions = round((df_realized_grouped['EP ID']/df_approved_grouped['EP ID'])*100,2).fillna(value=0)
    df_combined = pandas.concat([df_approved_grouped,df_realized_grouped,df_conversions],ignore_index=True, axis=1).fillna(value=0)
    return df_combined

# Generate Output

Throughout the generation of the report we will be filtering by each product and then generate the analysis for the whole and per product.

## Total

In [41]:
# result = create_report()
create_report().to_csv('Historical-conversion.csv')
# compute_growth_equation(result)

## Global Volunteer

In [9]:
create_report(product='GV',with_numerics=True).to_csv('Historical-conversion-gv.csv')

## Global Talent

In [10]:
create_report(product='GT',with_numerics=True).to_csv('Historical-conversion-gt.csv')

## Global Entrepreneur

In [11]:
create_report(product='GE',with_numerics=True).to_csv('Historical-conversion-ge.csv')

# Support Functions

These are additional functions that enable us to keep complying with the DRY Principle and not repeat ourselves every now and then.

In [8]:
def create_report(product=None,with_numerics=False):
    df_1516_copy = df_1516.copy()
    df_1617_copy = df_1617.copy()
    df_1718_copy = df_1718.copy()
    df_1819_copy = df_1819.copy()
    if product: 
        df_1516_copy = filter_product(df_1516_copy,product)
        df_1617_copy = filter_product(df_1617_copy,product)
        df_1718_copy = filter_product(df_1718_copy,product)
        df_1819_copy = filter_product(df_1819_copy,product)
    if with_numerics:
        return pandas.concat([conversion_rates(df_1516_copy,2016,8,31),
                   conversion_rates(df_1617_copy,2017,8,31),
                  conversion_rates(df_1718_copy,2018,8,31),
                    conversion_rates(df_1819_copy)[0]
                  ],ignore_index=True, axis=1)
    else :
        return pandas.concat([conversion_rates(df_1516_copy,2016,8,31)[2],
                   conversion_rates(df_1617_copy,2017,8,31)[2],
                  conversion_rates(df_1718_copy,2018,8,31)[2],
                  conversion_rates(df_1819_copy)[0]
                  ],ignore_index =True, axis=1)

In [6]:
def filter_product(data,product):
    return data.loc[data['Product'] == product]

In [7]:
def compute_growth_equation(data):
    return data.concat([data,data[1]],axis=1)